In [21]:
%load_ext autoreload

from pathlib import Path

from tqdm import tqdm, trange

import numpy as np

from scipy import ndimage as ndi

import skimage
import skimage.feature
import skimage.filters
import skimage.morphology
import skimage.segmentation as seg
import skimage.measure

import cv2 as cv

from bokeh.plotting import output_notebook, figure, show
from bokeh.layouts import column, row, layout, gridplot
from bokeh.models import ColumnDataSource, Slider

import fish

output_notebook()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading BokehJS ...

In [2]:
THIS_DIR = Path.cwd()
ROOT_DIR = THIS_DIR.parent
DATA_DIR = ROOT_DIR / 'data'

In [3]:
def show_frame(frame):
    W = frame.shape[1]
    H = frame.shape[0]

    def _(doc):
        p = figure(match_aspect = True)
        p.x_range.range_padding = p.y_range.range_padding = 0

        source = ColumnDataSource(data=dict(image=[frame[:, :]]))

        p.image(image='image', source = source, x = 0, y = 0, dw = W, dh = H, palette="Greys256")

        doc.add_root(p)
        
    return show(_)

def show_frames(frames):
    N = frames.shape[0]
    W = frames.shape[2]
    H = frames.shape[1]

    def _(doc):
        p = figure(match_aspect = True)
        p.x_range.range_padding = p.y_range.range_padding = 0

        source = ColumnDataSource(data=dict(image=[frames[0, :, :]]))

        slider = Slider(start=0, end=(N-1), value=0, step=1, title="Frame", callback_policy='throttle', callback_throttle=100)
        
        def update(attr, old, new):
            source.data = dict(image=[frames[new, :, :]])
        
        slider.on_change('value_throttled', update)

        p.image(image='image', source = source, x = 0, y = 0, dw = W, dh = H, palette="Greys256")

        doc.add_root(column(p, slider))
        
    return show(_)

In [7]:
movie = fish.read(DATA_DIR / 'D1-1_17_medium.mp4')
movie = np.fliplr(movie)
print(movie.shape)

Reading frames from C:\Users\joshk\projects\fish\data\D1-1_17_medium.mp4: 100%|██████████████████████████████████████████████████████████████████████████████████████| 1816/1816 [00:09<00:00, 199.72it/s]

(1816, 1024, 1280)


In [6]:
show_frames(movie)

In [ ]:
backsub = cv.createBackgroundSubtractorMOG2()
backsubbed_movie = np.empty_like(movie)
for frame_idx, frame in enumerate(tqdm(movie)):
    backsubbed_movie[frame_idx] = backsub.apply(frame)

In [ ]:
show_frames(backsubbed_movie)

In [ ]:
show_frame(backsub.getBackgroundImage())

In [ ]:
avg = np.mean(movie, axis=0)
show_frame(avg)

In [11]:
shuffle_backsub = cv.createBackgroundSubtractorMOG2(detectShadows=False)

shuffled = movie.copy()

for _ in trange(1):
    np.random.shuffle(shuffled)
    for frame in shuffled:
        shuffle_backsub.apply(frame)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:36<00:00, 36.23s/it]


In [12]:
show_frame(shuffle_backsub.getBackgroundImage())

In [14]:
backsubbed_movie = np.empty_like(movie)
for frame_idx, frame in enumerate(tqdm(movie)):
    backsubbed_movie[frame_idx] = shuffle_backsub.apply(frame, learningRate = 0)  # don't update the background model, which we've pre-trained


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1816/1816 [00:47<00:00, 38.54it/s]


In [15]:
show_frames(backsubbed_movie)

In [17]:
def canny(frames):
    N = frames.shape[0]
    W = frames.shape[2]
    H = frames.shape[1]

    def _(doc):
        orig = figure(match_aspect = True)
        cann = figure(match_aspect = True)
        orig.x_range.range_padding = orig.y_range.range_padding = 0
        cann.x_range = orig.x_range
        cann.y_range = orig.y_range
        
        frame_opts = {
            'index': 0
        }
        canny_opts = {
            'sigma': 1,
            'low_threshold': 10,
            'high_threshold': 200,
        }

        orig_source = ColumnDataSource(data=dict(image=[frames[frame_opts['index'], :, :]]))
        cann_source = ColumnDataSource(data=dict(image=[skimage.feature.canny(frames[frame_opts['index'], :, :], **canny_opts)]))

        frame_slider = Slider(start=0, end=(N-1), value=frame_opts['index'], step=1, title="Frame", callback_policy='mouseup', callback_throttle=100)
        sigma_slider = Slider(start=0, end=3, value=canny_opts['sigma'], step=.01, title="Sigma", callback_policy='mouseup', callback_throttle=100)
        low_slider = Slider(start=0, end=255, value=canny_opts['low_threshold'], step=.01, title="Low Threshold", callback_policy='mouseup', callback_throttle=100)
        high_slider = Slider(start=0, end=255, value=canny_opts['high_threshold'], step=.01, title="High Threshold", callback_policy='mouseup', callback_throttle=100)
        
        def updater(opt_dict, key):
            def _(attr, old, new):
                opt_dict[key] = new
                
                f = frames[frame_opts['index']]
                orig_source.data = dict(image=[f])
                
                c = skimage.feature.canny(f, **canny_opts)
                cann_source.data = dict(image=[c])
                
            return _
        
        frame_slider.on_change('value_throttled', updater(frame_opts, 'index'))
        sigma_slider.on_change('value_throttled', updater(canny_opts, 'sigma'))
        low_slider.on_change('value_throttled', updater(canny_opts, 'low_threshold'))
        high_slider.on_change('value_throttled', updater(canny_opts, 'high_threshold'))

        orig.image(image='image', source = orig_source, x = 0, y = 0, dw = W, dh = H, palette="Greys256")
        cann.image(image='image', source = cann_source, x = 0, y = 0, dw = W, dh = H, palette="Greys256")

        doc.add_root(column(row(orig, cann), frame_slider, row(sigma_slider, low_slider, high_slider)))
        
    return show(_)

In [ ]:
canny(movie)

In [ ]:
frame = movie[0]

markers = np.zeros_like(frame)
markers[frame < 20] = 0
markers[frame > 100] = 2

show_frame(markers)

In [ ]:
elevation = skimage.filters.scharr(frame)
show_frame(elevation)

In [ ]:
segmentation = skimage.morphology.watershed(elevation, markers, compactness = .0001)
show_frame(segmentation)

In [18]:
canny(backsubbed_movie)

In [19]:
c = skimage.feature.canny(backsubbed_movie[186], low_threshold=26, high_threshold=239, sigma=1.5)

In [20]:
show_frame(c)

In [32]:
contours = skimage.measure.find_contours(c, .5)

In [33]:
len(contours)

564

In [41]:
filled = ndi.binary_fill_holes(ndi.binary_closing(c))
show_frame(filled)

In [46]:
labelled, num = ndi.label(c)
show_frame(labelled)